Links

more comments error - https://www.reddit.com/r/redditdev/comments/5kwxb3/morecomments_object_attribute_error/ 
ML sentiment analysis - https://www.datarobot.com/blog/using-machine-learning-for-sentiment-analysis-a-deep-dive/#:~:text=Convolutional%20neural%20networks,used%20in%20computer%20vision%20models. 
Simple VADER sent analysis - https://medium.com/bitgrit-data-science-publication/sentiment-analysis-on-reddit-tech-news-with-python-cbaddb8e9bb6 


Imports

In [69]:
pip install transformers 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [70]:
import pandas as pd
import numpy as np

# misc
import datetime as dt
from pprint import pprint
from itertools import chain

# reddit crawler
import praw

# sentiment analysis
# import nltk
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
# from nltk.corpus import stopwords

# import nltk
# import ssl

# nltk.download('vader_lexicon') # get lexicons data
# nltk.download('punkt') # for tokenizer
# nltk.download('stopwords')

#BERT sentiment analysis
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

r = praw.Reddit(user_agent='DemApollus',
                client_id='KrnUePjuO_J50XuOWNIyRw',
                client_secret='OswzYFW1nBcQWSfZPQVHmOXRglUcvw',
                check_for_async=False)

In [71]:
''' 
sid = SentimentIntensityAnalyzer()

pos_text = "please take this class if u dont want to miss out on death!"
cap_pos_text = "Vader is AWESOME!" # captilization and ! increases the effect
neg_text = "Vader is bad"

print(sid.polarity_scores(pos_text))
print(sid.polarity_scores(cap_pos_text))
print(sid.polarity_scores(neg_text)) 
'''

' \nsid = SentimentIntensityAnalyzer()\n\npos_text = "please take this class if u dont want to miss out on death!"\ncap_pos_text = "Vader is AWESOME!" # captilization and ! increases the effect\nneg_text = "Vader is bad"\n\nprint(sid.polarity_scores(pos_text))\nprint(sid.polarity_scores(cap_pos_text))\nprint(sid.polarity_scores(neg_text)) \n'

In [72]:
course_code = "CPSC 121"

In [73]:
ubc_reddit = r.subreddit("UBC")
titles = list(())
urls = list(())
comments = list(())
for post in ubc_reddit.search(course_code, limit=20):
    titles.append(post.title)
    urls.append(post.url)
    submission = r.submission(id=post.id)
    for top_level_comment in submission.comments:
        comm = top_level_comment.body
        if (len(comm) > 10 and len(comm) < 150):
            comments.append(comm)
    #for comment in submission.comments.list():
        #print(post.title)
        #comments.append(comment.body)
        #print(comm)
#print(urls[0])

In [74]:
#perform sentiment analysis using BERT
BSA_comments = list(())
for i in comments:
    tokens = tokenizer.encode(i, return_tensors='pt')
    result = model(tokens)
    result.logits
    BSA_comments.append(int(torch.argmax(result.logits))+1)


In [75]:
#for j in range(len(BSA_comments)):
    #print(str(BSA_comments[j]) + ": " + comments[j])

In [76]:
df = pd.DataFrame({"Comments": comments, "BERT Ranking": BSA_comments})
df.to_csv(course_code + ".csv")